In [ ]:
import sys
import os
from PyQt5 import QtWidgets, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QWidget, QVBoxLayout, QFileDialog
from PyQt5.QtGui import QPixmap, QFont

class MyWindow(QMainWindow):
    def __init__(self):
        super(MyWindow, self).__init__()
        self.setGeometry(660, 240, 500, 500)
        self.setWindowTitle("SIMA (Speech Intelligibility Mosque Analyzer)")
        self.setStyleSheet("background-color: white;")
        self.initUI()

    def initUI(self):
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)

        layout = QVBoxLayout()
        layout.setAlignment(QtCore.Qt.AlignTop)

        # Path relatif logo (bisa diganti sesuai kebutuhan)
        img_path2 = os.path.join("C:/Users/ndaru/OneDrive - KFUPM/THESIS/DATASET/Apps/kfupm logo.png")
        pixmap2 = QPixmap(img_path2).scaled(75, 75, QtCore.Qt.KeepAspectRatio, QtCore.Qt.SmoothTransformation)

        self.lbl_img = QLabel(self)
        self.lbl_img.setPixmap(pixmap2)
        self.lbl_img.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_img, alignment=QtCore.Qt.AlignHCenter)

        layout.addSpacing(10)

        self.lbl_dept = QLabel("Architectural Engineering and Construction Management Department", self)
        self.lbl_dept.setFont(QFont("Arial", 8))
        self.lbl_dept.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_dept)

        self.lbl_college = QLabel("College of Design and Built Environment", self)
        self.lbl_college.setFont(QFont("Arial", 8))
        self.lbl_college.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_college)

        self.lbl_university = QLabel("King Fahd University of Petroleum and Minerals", self)
        self.lbl_university.setFont(QFont("Arial", 8))
        self.lbl_university.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_university)

        layout.addSpacing(50)

        self.lbl_welcome = QLabel("Welcome to:", self)
        self.lbl_welcome.setFont(QFont("Arial", 8))
        self.lbl_welcome.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_welcome)

        layout.addSpacing(30)

        self.lbl_sima = QLabel("SIMA", self)
        self.lbl_sima.setFont(QFont("Eras Demi ITC", 35))
        self.lbl_sima.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_sima)

        self.lbl_simal = QLabel("Speech Intelligibility Mosque Analyzer", self)
        self.lbl_simal.setFont(QFont("Eras Demi ITC", 15))
        self.lbl_simal.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_simal)

        layout.addSpacing(50)

        self.btn_browse = QtWidgets.QPushButton("Browse Impulse Response (.wav) File", self)
        self.btn_browse.setFont(QFont("Arial", 9))
        self.btn_browse.clicked.connect(self.browse_file)
        layout.addWidget(self.btn_browse)

        self.lbl_path = QtWidgets.QLabel("Selected file will appear here", self)
        self.lbl_path.setFont(QFont("Arial", 9))
        self.lbl_path.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_path)

        layout.addSpacing(30)

        self.btn_predict = QtWidgets.QPushButton("Predict", self)
        self.btn_predict.setFont(QFont("Arial", 9))
        self.btn_predict.clicked.connect(self.predict)
        layout.addWidget(self.btn_predict)

        self.lbl_result = QtWidgets.QLabel("Result will appear here", self)
        self.lbl_result.setFont(QFont("Arial", 9, QFont.Bold))
        self.lbl_result.setAlignment(QtCore.Qt.AlignCenter)
        layout.addWidget(self.lbl_result)

        central_widget.setLayout(layout)

    def browse_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Select a WAV file", "", "WAV Files (*.wav)")
        if file_path:
            self.lbl_path.setText(file_path)
        else:
            self.lbl_path.setText("No file selected.")

    def predict(self):
        import numpy as np
        import librosa
        import soundfile as sf
        import tensorflow as tf
        from scipy.signal import butter, sosfilt

        SR = 22050
        N_MFCC = 13
        N_FFT = 2048
        HOP_LENGTH = 128
        DURATION = 4.0
        SAMPLES_PER_TRACK = int(SR * DURATION)

        from keras.layers import TFSMLayer
        from keras import Sequential
        
        model_path = "C:/Users/ndaru/OneDrive - KFUPM/THESIS/DATASET/Apps/cnn_audio_model2"
        layer = TFSMLayer(model_path, call_endpoint="serve")
        model = Sequential([layer])

        def bandpass_filter(signal, sr, lowcut=250, highcut=2000, order=5):
            sos = butter(order, [lowcut, highcut], btype='band', fs=sr, output='sos')
            return sosfilt(sos, signal)

        def extract_mfcc_stereo(file_path):
            signal, sample_rate = librosa.load(file_path, sr=SR, mono=False)

            if signal.shape[1] < SAMPLES_PER_TRACK:
                pad_length = int(SAMPLES_PER_TRACK - signal.shape[1])
                signal = np.pad(signal, ((0, 0), (0, pad_length)), mode="constant")
            elif signal.shape[1] > SAMPLES_PER_TRACK:
                signal = signal[:, :SAMPLES_PER_TRACK]

            left_channel = bandpass_filter(signal[0], sample_rate)
            right_channel = bandpass_filter(signal[1], sample_rate)

            mfcc_left = librosa.feature.mfcc(y=left_channel, sr=SR, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH).T
            mfcc_right = librosa.feature.mfcc(y=right_channel, sr=SR, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH).T

            mfcc_left = (mfcc_left - np.mean(mfcc_left, axis=0)) / (np.std(mfcc_left, axis=0) + 1e-10)
            mfcc_right = (mfcc_right - np.mean(mfcc_right, axis=0)) / (np.std(mfcc_right, axis=0) + 1e-10)

            feature = np.stack((mfcc_left, mfcc_right), axis=-1)
            feature = feature[..., np.newaxis]

            return feature

        def classify(file_path):
            X = extract_mfcc_stereo(file_path)
            X = np.expand_dims(X, axis=0)

            prediction = model.predict(X)
            predicted_label = (prediction[:, 1] > 0.5).astype(int)
            
            class_names = ["High Intelligibility", "Low Intelligibility"]
            predicted_label_name = class_names[int(predicted_label[0])]

            probabilities_percent = prediction[0] * 100
            prob_string = ", ".join([f"{class_names[i]}: {probabilities_percent[i]:.1f}%" for i in range(len(class_names))])

            self.lbl_result.setText(f"Predicted class: {predicted_label_name} ({prob_string})")
            return predicted_label_name

        audio_file = self.lbl_path.text()
        if os.path.exists(audio_file):
            classify(audio_file)
        else:
            self.lbl_result.setText("No valid file selected.")

def window():
    app = QApplication(sys.argv)
    win = MyWindow()
    win.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    window()